In [ ]:
# Implementation of Masked Letter Guessing Game.

In [1]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
from torch.utils.data import DataLoader, Dataset

from transformers import CanineTokenizer, CanineModel, Trainer, TrainingArguments
import torch.nn as nn
# import datasets

# # Define your list of words
# words = ["apple", "banana", "orange", "pineapple", "grape", "watermelon"]




In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import shutil

# Source path in Google Drive
source_path = '../words_250000_train.txt'

# Destination path in Colab environment
destination_path = '/words_250000_train.txt'

# Copy the file from Google Drive to Colab
shutil.copyfile(source_path, destination_path)

'/words_250000_train.txt'

In [4]:
# Load data from the text file
data_file ="/words_250000_train.txt"
with open(data_file, "r") as file:
    words = [line.strip() for line in file]

In [5]:
words = [" ".join(list(word)) for word in words]

In [6]:
len(words), words[1]


(227300, 'a a a a a a')

In [7]:
# Tokenize the words on character level
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",vocab_file="vocab.txt")
# tokenizer = CanineTokenizer.from_pretrained('google/canine-s',model_max_length=200) #,model_max_length=512
tokenized_words = tokenizer(words, padding=True, truncation=True, return_tensors="pt", is_split_into_words=True )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# Get vocab size from the tokenizer
vocab_size = tokenizer.vocab_size
vocab_size

57

In [9]:
# Define a custom dataset
class MaskedLetterDataset(Dataset):
    def __init__(self, tokenized_words, tokenizer):
        self.tokenized_words = tokenized_words
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.tokenized_words["input_ids"])

    def __getitem__(self, idx):
        input_ids = self.tokenized_words["input_ids"][idx].clone()
        label_ids = input_ids.clone()

        # Randomly choose a position to mask
        mask_index = torch.randint(1, input_ids.shape[0], (1,)).item()
        # Mask the chosen position
        input_ids[mask_index] = tokenizer.mask_token_id

        return input_ids, label_ids



In [26]:
# Create dataset and dataloader
dataset = MaskedLetterDataset(tokenized_words, tokenizer)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize pre-trained BERT model for MLM
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
# model = CanineModel.from_pretrained('google/canine-s')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
# Fine-tune the model
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
for epoch in range(num_epochs):
    for batch in dataloader:
        input_ids, label_ids = batch
        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, labels=label_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

# Save the fine-tuned model
model.save_pretrained("fine_tuned_model")



Epoch 1/1000, Loss: 8.99145221710205
Epoch 2/1000, Loss: 7.057374477386475
Epoch 3/1000, Loss: 6.566317081451416
Epoch 4/1000, Loss: 6.236056804656982
Epoch 5/1000, Loss: 5.9101433753967285
Epoch 6/1000, Loss: 5.604120254516602
Epoch 7/1000, Loss: 5.311336040496826
Epoch 8/1000, Loss: 5.068897724151611
Epoch 9/1000, Loss: 4.857880115509033
Epoch 10/1000, Loss: 4.667656898498535
Epoch 11/1000, Loss: 4.496009826660156
Epoch 12/1000, Loss: 4.345120429992676
Epoch 13/1000, Loss: 4.211848258972168
Epoch 14/1000, Loss: 4.087759017944336
Epoch 15/1000, Loss: 3.967772960662842
Epoch 16/1000, Loss: 3.858030080795288
Epoch 17/1000, Loss: 3.751708507537842
Epoch 18/1000, Loss: 3.654512643814087
Epoch 19/1000, Loss: 3.561464309692383
Epoch 20/1000, Loss: 3.472590446472168
Epoch 21/1000, Loss: 3.389397144317627
Epoch 22/1000, Loss: 3.3104944229125977
Epoch 23/1000, Loss: 3.234757661819458
Epoch 24/1000, Loss: 3.162339925765991
Epoch 25/1000, Loss: 3.0939877033233643
Epoch 26/1000, Loss: 3.028728246

In [14]:
# Inference
def predict_masked_letter(word):
    tokenized_word = tokenizer(word, return_tensors="pt", is_split_into_words=True)
    mask_index = torch.where(tokenized_word["input_ids"][0] == tokenizer.mask_token_id)

    with torch.no_grad():
        outputs = model(**tokenized_word)
        print(outputs)
        predictions = torch.argmax(outputs.logits[0, mask_index]).item()
        predicted_letter = tokenizer.convert_ids_to_tokens(predictions)

    return predicted_letter


In [19]:
#abaptiston
word = f"[MASK]onsoon"
tokenized_word = tokenizer(word, return_tensors="pt")
# Find the index of the masked token
mask_index = torch.where(tokenized_word["input_ids"][0] == tokenizer.mask_token_id)[0].item()

In [22]:
# Get the model's output
with torch.no_grad():
    outputs = model(**tokenized_word)

# Extract logits for the masked token
mask_token_logits = outputs.logits[0, mask_index]

# Get the top 10 predictions
top_20_predictions = torch.topk(mask_token_logits, 20).indices

# Convert the token IDs to tokens (letters)
top_20_tokens = [tokenizer.convert_ids_to_tokens(pred.item()) for pred in top_20_predictions]
print(top_20_tokens)
top_20_tokens = [pred.replace("#","") for pred in top_20_tokens]
print(top_20_tokens)
top_20_tokens=list(dict.fromkeys(top_20_tokens))
print(top_20_tokens)
letters = " ".join(list(word)).split(" ")
top_words = [let for let in top_20_tokens if let not in letters]
print(top_words)

['a', 'r', 'o', 's', 'n', 'l', 'i', 'u', 'b', 'c', 'd', 'e', 't', 'k', 'h', 'm', 'g', 'p', 'v', 'f']
['a', 'r', 'o', 's', 'n', 'l', 'i', 'u', 'b', 'c', 'd', 'e', 't', 'k', 'h', 'm', 'g', 'p', 'v', 'f']
['a', 'r', 'o', 's', 'n', 'l', 'i', 'u', 'b', 'c', 'd', 'e', 't', 'k', 'h', 'm', 'g', 'p', 'v', 'f']
['a', 'r', 'l', 'i', 'u', 'b', 'c', 'd', 'e', 't', 'k', 'h', 'm', 'g', 'p', 'v', 'f']
